In [1]:
%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


In [20]:
verticies_text =\
"0.0000 0.0000 -0.0000 "+\
"1.0000 0.0000 -0.0000 "+\
"0.0000 0.0000 -1.0000 "+\
"1.0000 0.0000 -1.0000 "+\
"0.0000 1.0000 -0.0000 "+\
"1.0000 1.0000 -0.0000 "+\
"0.0000 1.0000 -1.0000 "+\
"1.0000 1.0000 -1.0000 "

faces_text = \
"1 3 4 "+\
"4 2 1 "+\
"5 6 8 "+\
"8 7 5 "+\
"1 2 6 "+\
"6 5 1 "+\
"2 4 8 "+\
"8 6 2 "+\
"4 3 7 "+\
"7 8 4 "+\
"3 1 5 "+\
"5 7 3 "

vertecies = np.array( [ float(i) for i in verticies_text.split() ] )
faces     = np.array( [ int(i) for i in faces_text.split() ]).reshape(-1,3)


In [151]:

def tostr(arg):
    if isinstance(arg,list):
        return ", ".join( [ tostr(i) for i in arg] )
    else:
        return '"{}"'.format(arg) if isinstance(arg,str) else str(arg)

    
class FBXNode:
    
    def __init__(self,typename,**kwargs):
        self.typename = typename
        self.properties = kwargs.get("prop",[])
        self.subnodes = kwargs.get("sub",[])
    
    
    def write(self, *args,**kwargs):
        for i in range(self.indent_size):
            self.file.write("\t")
        self.file.write(*args,**kwargs)
    
    def writeln(self, *args, **kwargs):
        if len(args) > 0:
            self.write(*args,**kwargs)
        self.file.write("\n")
        
    def addProperty( self, prop ):
        if not isinstance(self.properties,list):
            self.properties = [self.properties]
        self.properties.append(prop)
        
    def addNode( self, *args ):
        if len(args) == 0:
            return
        
        if isinstance(args[0],FBXNode):
            self.subnodes.append(args[0])
        else:
            if len(args) == 2:
                newNode = FBXNode(args[0],prop = args[1])
                self.subnodes.append(newNode)
            
        
    def serialize( self, file, *args, **kwargs):
        self.file = file
        self.indent_size = kwargs.get("indent_size",0)
        
        #write typename
        self.write("{}: ".format(self.typename))
        
        #write name properties
        file.write("{}".format(tostr(self.properties)))
        
        #write subnodes
        if not (self.subnodes is None) and len(self.subnodes) > 0:
            self.writeln(" {")
            for node in self.subnodes:
                node.serialize(file,indent_size = self.indent_size+1)
            self.writeln("}")
        else:
            self.writeln()

In [153]:
node = FBXNode("Test")
node.addProperty("Fuck")
node.addProperty(10)
node.addNode("Fuck me",[1,2,3,4,"BITCH"])

node2 = FBXNode("Test2")
node2.addProperty("G")
node2.addProperty(20)
node2.addProperty(3)

node.addNode(node2)

with open("test.fbx","w") as f:
    node.serialize(f)